# A common interface for handling tabular data

As we've seen in the FITS tutorial, the [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) sub-package can be used to access FITS tables. In addition, as we will see in the next tutorial, there is functionality in [astropy.io.votable](http://docs.astropy.org/en/stable/io/votable/index.html) and [astropy.io.ascii](http://docs.astropy.org/en/stable/io/ascii/index.html) to read in VO and ASCII tables. However, while these sub-pacakges have user interfaces that are specific to each kind of file, it can be difficult to remember all of them. Therefore, astropy includes a higher level interface in [astropy.table](http://docs.astropy.org/en/stable/table/index.html) which can be used to access tables in many different formats in a similar way.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create tables</li>
<li>Access data in tables</li>
<li>Combining tables</li>
<li>Using high-level objects as columns</li>
<li>Aggregation</li>
<li>Masking</li>
<li>Reading/writing</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.table. For more information about the features presented below as well as other available features, you can read the
[astropy.table documentation](https://docs.astropy.org/en/stable/table/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating tables

The main class we will use here is called ``Table``:

In [ ]:
from astropy.table import Table

Before we look at how to read and write tables, let's first see how to create a table from scratch:

In [ ]:
t1 = Table()
t1['name'] = ['source1', 'source2', 'source3']
t1['flux'] = [1.1, 1.2, 1.3]

We can look at the table with:

In [ ]:
t1

We can add columns:

In [ ]:
t1['size'] = [1, 3, 6]
t1

Access the values in a column:

In [ ]:
t1['size']

In [ ]:
t1['flux'][1]

Convert the column to a Numpy array:

In [ ]:
import numpy as np
np.array(t1['size'])

Access individual cells:

And access rows:

In [ ]:
t1['size']>1

## Units in tables

Table columns can include units:

In [ ]:
from astropy import units as u
t1['size'].unit = u.cm
t1['flux'].unit = 'mJy'
t1

Some unitful operations will then work:

In [ ]:
t1['size'].to('m')

In [ ]:
type(t1['size'].to('m'))

In [ ]:
type(t1['size'])

However, you may run into unexpected behavior, so if you are planning on using table columns as Quantities, we recommend that you use the ``QTable`` class:

In [ ]:
t1

In [ ]:
from astropy.table import QTable
qt1 = QTable(t1)

In [ ]:
qt1

In [ ]:
type(qt1['size'])


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Make a table that contains three columns: <code>spectral type</code>, <code>temperature</code>, and <code>radius</code>, and incude 5 rows with fake data (or real data if you like, for example from <a href="http://www.atlasoftheuniverse.com/startype.html">here</a>). Try including units on the columns that can have them.</li>
<li>Find the mean temperature and the maximum radius</li>
<li>Try and find out how to add and remove rows</li>
<li>Add a new column which gives the luminosity (using $L=4\pi R^2 \sigma T^4$)</li>
</ol>

</div>

</section>


## Iterating over tables

It is possible to iterate over rows or over columns. To iterate over rows, iterate over the table itself:

In [ ]:
for row in t1:
    print(row)

Rows can act like dictionaries, so you can access specific columns from a row:

In [ ]:
for row in t1:
    print(row['size'], type(row['size']))

In [ ]:
for row in qt1:
    print(row['size'], type(row['size']))

Iterating over columns is also easy:

In [ ]:
for colname in t1.columns:
    column = t1[colname]
    print(column[0])

Accessing specific rows from a column object can be done with the item notation:

## Joining tables

The astropy.table sub-package provides a few useful functions for stacking/combining tables. For example, we can do a 'join':

In [ ]:
add: https://docs.astropy.org/en/stable/table/operations.html#table-operations

In [ ]:
t2 = Table()
t2['name'] = ['source1', 'source3']
t2['flux2'] = [1, 9]

In [ ]:
from astropy.table import join

In [ ]:
t3 = join(t1, t2, join_type='outer')
t3

In [ ]:
np.mean(t3['flux2'])

## Masked tables

It is possible to mask individual cells in tables:

In [ ]:
t4 = Table(masked=True)

In [ ]:
t4['id'] = [4, 5, 6]
t4['flux'] = [1.3, 1.5, 1.6]

In [ ]:
t4

In [ ]:
t4['flux'].mask = [1, 0, 1]
t4

In [ ]:
select = np.array([False, True, True])

In [ ]:
t5 = t4[select]

## Using high-level objects as columns

A few specific astropy high-level objects can be used as columns in table - this includes SkyCoord and Time:

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [ ]:
t6 = Table()

In [ ]:
t6['time'] = Time([50000, 51000, 52000], format='mjd')

In [ ]:
t6['coord'] = SkyCoord([1, 2, 3] * u.deg, [4, 5, 6] * u.deg)

In [ ]:
t6['flux'] = [1, 5, 4] * u.mJy

In [ ]:
t6

In [ ]:
t6[0]['coord']

Note however that you may not necessarily be able to write this table to a file and get it back intact, since being able to store this kind of information is not possible in all file formats.

## Slicing

Tables can be sliced like Numpy arrays:

In [ ]:
obs = Table(rows=[('M31' , '2012-01-02', 17.0, 17.5),
                  ('M31' , '2012-01-02', 17.1, 17.4),
                  ('M101', '2012-01-02', 15.1, 13.5),
                  ('M82' , '2012-02-14', 16.2, 14.5),
                  ('M31' , '2012-02-14', 16.9, 17.3),
                  ('M82' , '2012-02-14', 15.2, 15.5),
                  ('M101', '2012-02-14', 15.0, 13.6),
                  ('M82' , '2012-03-26', 15.7, 16.5),
                  ('M101', '2012-03-26', 15.1, 13.5),
                  ('M101', '2012-03-26', 14.8, 14.3)],
            names=['name', 'obs_date', 'mag_b', 'mag_v'])

In [ ]:
obs

In [ ]:
obs[1:4]

In [ ]:
obs[obs['mag_b'] > 15.5]

In [ ]:
obs['mag_b', 'mag_v']


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Starting from the <code>obs</code> table:</p>
<ol>
<li>Make a new table that shows every other row, starting with the second row? (that is, the second, fourth, sixth, etc. rows).</li>
<li>Make a new table the only contains rows where <code>name</code> is <code>M31</code></li>
</ol>

</div>

</section>


## Grouping and Aggregation

It is possible to aggregate rows of a table together - for example, to group the rows by source name in the ``obs`` table, you can do:

In [ ]:
obs_by_name = obs.group_by('name')

In [ ]:
obs_by_name

This is not just sorting the values but actually making it possible to access each group of rows:

In [ ]:
for group in obs_by_name.groups:
    print(group)
    print("")

In [ ]:
obs_by_name.groups

We can then aggregate the rows together in each group using a function:

In [ ]:
obs_by_name.groups.aggregate(np.mean)

In [ ]:
obs_by_name['name', 'mag_v'].groups.aggregate(np.mean)

## Writing data

To write out the data, we can use the ``write`` method:

In [ ]:
obs

In [ ]:
obs.write('test.fits')

In [ ]:
obs.write('test.tex')

In some cases the format will be inferred from the extension, but only in unambiguous cases - otherwise the format has to be specified explicitly:

In [ ]:
obs.write('test.vot', format='votable', overwrite=True)

You can find the [list of supported formats](https://docs.astropy.org/en/stable/io/unified.html#built-in-table-readers-writers) in the documentation.

## Reading data

You can also easily read in tables using the ``read`` method:

In [ ]:
t6 = Table.read('data/2mass.tbl', format='ascii.ipac')

In [ ]:
Table.read.help('ascii')

In [ ]:
Table.read('data/gaia_lmc_psc.fits')


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Using the <code>t6</code> table above:</p>
<ol>
<li>
<p>Make a plot that shows <code>j_m</code>-<code>h_m</code> on the x-axis, and <code>h_m</code>-<code>k_m</code> on the y-axis</p>
</li>
<li>
<p>Make a new table that contains the subset of rows where the <code>j_snr</code>, <code>h_snr</code>, and <code>k_snr</code> columns, which give the signal-to-noise-ratio in the J, H, and K band, are greater than 10, and try and show these points in red in the plot you just made.</p>
</li>
<li>
<p>Make a new table (based on the full table) that contains only the RA, Dec, and the <code>j_m</code>, <code>h_m</code> and <code>k_m</code> columns, then try and write out this catalog into a format that you can read into another software package. For example, try and write out the catalog into CSV format, then read it into a spreadsheet software package (e.g. Excel, Google Docs, Numbers, OpenOffice). </p>
</li>
</ol>

</div>

</section>


<center><i>This notebook was originally written by <a href="https://aperiosoftware.com/">Aperio Software Ltd</a> and modified by Brigitta Sipőcz. &copy; 2019-2021 and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)